#### Text Generation With LSTM Recurrent Neural Networks in Python with Keras

In [6]:
# importing libraries
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import sys

#### to load the sql_logs into memory and convert all of the characters to lowercase to reduce the vocabulary the network must learn.

In [7]:

# load ascii text and covert to lowercase
filename = r"C:\Users\lokes\OneDrive\Desktop\optacloud_assignment_lokesh naik\ML_model\sql_logs.csv"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  80366
Total Vocab:  49


#### 80366 characters, and when converted to lowercase, there are only 49 distinct characters in the vocabulary for the network to learn

#### split the sql_logs into sequences, converting the characters to integers

In [8]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
 seq_in = raw_text[i:i + seq_length]
 seq_out = raw_text[i + seq_length]
 dataX.append([char_to_int[char] for char in seq_in])
 dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  80266


#### Running the code to this point shows that when you split up the dataset into training data for the network to learn that you have just under 80366 training patterns. This makes sense as, excluding the first 100 characters, you have one training pattern to predict each of the remaining characters.

In [9]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

#### defining the LSTM model. Here, defining a single hidden LSTM layer with 256 memory units. The network uses dropout a probability of 20. The output layer is a Dense layer using the softmax activation function to output a probability prediction for each of the 49 characters between 0 and 1.

In [10]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
# define the checkpoint
filepath=r"C:\Users\lokes\OneDrive\Desktop\optacloud_assignment_lokesh naik\ML_model\weights\weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
# Here, a modest number of 10 epochs and a large batch size of 128 patterns
model.fit(X, y, epochs=10, batch_size=128, callbacks=callbacks_list)



Epoch 1/10


628/628 [==============================] - ETA: 0s - loss: 3.1340 - accuracy: 0.1357
Epoch 1: loss improved from inf to 3.13400, saving model to C:\Users\lokes\OneDrive\Desktop\optacloud_assignment_lokesh naik\ML_model\weights\weights-improvement-01-3.1340.hdf5
628/628 [==============================] - 166s 260ms/step - loss: 3.1340 - accuracy: 0.1357
Epoch 2/10
628/628 [==============================] - ETA: 0s - loss: 2.2852 - accuracy: 0.3302
Epoch 2: loss improved from 3.13400 to 2.28516, saving model to C:\Users\lokes\OneDrive\Desktop\optacloud_assignment_lokesh naik\ML_model\weights\weights-improvement-02-2.2852.hdf5
628/628 [==============================] - 164s 261ms/step - loss: 2.2852 - accuracy: 0.3302
Epoch 3/10
628/628 [==============================] - ETA: 0s - loss: 1.7391 - accuracy: 0.4871
Epoch 3: loss improved from 2.28516 to 1.73908, saving model to C:\Users\lokes\OneDrive\Desktop\optacloud_assignment_lokesh naik\ML_model\weights\weights-improvemen

### The accuracy of model is accuracy: 0.8693

#### the results may vary given the stochastic nature of the algorithm or evaluation procedure, or differences in numerical precision. 

#### After running the code, the hdf5 file is generated of weight checkpoint files and save in the given directory ('C:\Users\lokes\OneDrive\Desktop\optacloud_assignment_lokesh naik\ML_model\weights') path file

### Generating text using the trained LSTM network is relatively straightforward.

#### load the saved hdf5 file and copy the path and paste in filename 

In [20]:
# load the network weights
filename = r"C:\Users\lokes\OneDrive\Desktop\optacloud_assignment_lokesh naik\ML_model\weights\weights-improvement-10-0.3966.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

#### LSTM model to making predictions first start with a seed sequence as input, generate the next characters, then updating the seed sequence to add the generated characters on the end and trim off the first characters. This process is repeated for as long as we want to predict new query

In [21]:

int_to_char = dict((i, c) for i, c in enumerate(chars))
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(400):
 x = np.reshape(pattern, (1, len(pattern), 1))
 x = x / float(n_vocab)
 prediction = model.predict(x, verbose=0)
 index = np.argmax(prediction)
 result = int_to_char[index]
 seq_in = [int_to_char[value] for value in pattern]
 sys.stdout.write(result)
 pattern.append(index)
 pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" mit 1"
show index from `classicmodels`.`customers`
delete from offices where city = 56 limit 1
delet "
e from productl where productcode = 23 limit 1
"uelect ordernumber, orddrdate, requireddate, shippeddate, status from orders limit 1"
"update orders set ordernumber = 90, orderdate = 50, requireddate = 27, shippeddate = 82, status = 17 where shiupeddate = 25 limit 1"
"update orders set ordernumber = 90, orderdate = 50, requireddate = 27, shippeddate = 82, status = 17 where shiupeddate = 25 limit 1
Done.


#### The generated text with the random seed